In [3]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [4]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Puerto Ayora,EC,2022-08-26 23:14:05,-0.7393,-90.3518,65.55,96,75,5.99
1,1,Praia,CV,2022-08-26 23:14:06,14.9215,-23.5087,80.51,77,47,5.77
2,2,Tautira,PF,2022-08-26 23:14:07,-17.7333,-149.1500,76.39,71,9,8.32
3,3,Huarmey,PE,2022-08-26 23:14:07,-10.0681,-78.1522,63.93,79,1,11.83
4,4,Bluff,NZ,2022-08-26 23:14:08,-46.6000,168.3333,46.80,81,100,7.25


In [5]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [25]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [24]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.

# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [10]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
1,1,Praia,CV,2022-08-26 23:14:06,14.9215,-23.5087,80.51,77,47,5.77
2,2,Tautira,PF,2022-08-26 23:14:07,-17.7333,-149.1500,76.39,71,9,8.32
8,8,Kapaa,US,2022-08-26 23:14:10,22.0752,-159.3190,84.52,68,40,10.36
14,14,Butaritari,KI,2022-08-26 23:14:13,3.0707,172.7902,81.63,77,25,13.78
17,17,Makakilo City,US,2022-08-26 23:10:09,21.3469,-158.0858,86.32,59,75,17.27
19,19,Hithadhoo,MV,2022-08-26 23:14:15,-0.6000,73.0833,80.40,74,100,9.55
21,21,Waseca,US,2022-08-26 23:14:17,44.0777,-93.5074,77.43,73,0,4.61
23,23,Kyaukse,MM,2022-08-26 23:14:17,21.6000,96.1333,75.36,93,100,6.58
24,24,Kieta,PG,2022-08-26 23:14:18,-6.2167,155.6333,82.17,76,53,6.08
27,27,Grand-Santi,GF,2022-08-26 23:14:19,4.2500,-54.3833,77.74,75,95,3.24


In [11]:
preferred_cities_df.count()

City_ID       201
City          201
Country       201
Date          201
Lat           201
Lng           201
Max Temp      201
Humidity      201
Cloudiness    201
Wind Speed    201
dtype: int64

In [12]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Praia,CV,80.51,14.9215,-23.5087,
2,Tautira,PF,76.39,-17.7333,-149.1500,
8,Kapaa,US,84.52,22.0752,-159.3190,
14,Butaritari,KI,81.63,3.0707,172.7902,
17,Makakilo City,US,86.32,21.3469,-158.0858,
19,Hithadhoo,MV,80.40,-0.6000,73.0833,
21,Waseca,US,77.43,44.0777,-93.5074,
23,Kyaukse,MM,75.36,21.6000,96.1333,
24,Kieta,PG,82.17,-6.2167,155.6333,
27,Grand-Santi,GF,77.74,4.2500,-54.3833,


In [13]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [14]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
# Grab the first hotel from the results and store the name.
 # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [16]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [26]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [27]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [28]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [29]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))